# Battle of the Neighborhoods - Which place should I choose? 
### --- Code

## Table of Contents
1. Download and Explore Dataset
2. Explore Neighborhoods in New York City
3. Analyze Each Neighborhood
4. Cluster Neighborhoods
5. Examine Clusters and Find Similar Neighborhoods in Manhattan
6. Rank the Similar Neighborhoods by Studio Rent Price

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [80]:
! pip install BeautifulSoup4 # beautifulsoup library to extract data from HTML
import bs4 as bs

import urllib.request  # help in opening URLs

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

distributed 1.21.8 requires msgpack, which is not installed.
Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 1. Download and Explore Dataset

### (1) Rent Data
Average Rent Data in Manhattan, NY by neighborhood from online sources: https://www.rentcafe.com/average-rent-market-trends/us/ny/manhattan/ The dataset includes average rent data for “Studio”, “1-bedroom”, “2-bedrooms”, “3-bedrooms” and “All-rentals” in Manhattan by neighborhood. We do web Scraping and transform data into a dataframe. As we only need 'Studio Rent' data for analysis, the dataframe only has 2 columns: **[‘Neighborhood', 'Studio Rent']**.

#### Get the table

In [81]:
website_url = urllib.request.urlopen('https://www.rentcafe.com/average-rent-market-trends/us/ny/manhattan/').read()
soup = bs.BeautifulSoup(website_url,'lxml')
mytable = soup.find('table',{'class':'market-trends', 'id':'MarketTrendsAverageRentTable'})
mytable

<table class="market-trends" id="MarketTrendsAverageRentTable">
<thead>
<tr>
<th class="average-rent" colspan="6">Average Rent</th>
</tr>
</thead>
<tbody>
<tr>
<th>Neighborhood</th>
<th>All rentals</th>
<th>Studio</th>
<th>1 Bed</th>
<th>2 Beds</th>
<th>3 Beds</th>
</tr>
<tr class="current-row">
<th>Washington Heights</th>
<td>$2,170</td>
<td>$1,678</td>
<td>$2,000</td>
<td>$2,600</td>
<td>$3,219</td>
</tr>
<tr class="current-row">
<th>East Harlem</th>
<td>$2,528</td>
<td>$1,895</td>
<td>$2,362</td>
<td>$3,521</td>
<td>$5,755</td>
</tr>
<tr class="current-row">
<th>Harlem</th>
<td>$2,783</td>
<td>$1,975</td>
<td>$2,660</td>
<td>$4,112</td>
<td>$6,765</td>
</tr>
<tr class="current-row">
<th>Ellis Island</th>
<td>$3,328</td>
<td>$2,929</td>
<td>$3,384</td>
<td>$3,793</td>
<td>$1,424</td>
</tr>
<tr class="current-row">
<th>Tudor City</th>
<td>$3,389</td>
<td>$2,967</td>
<td>$3,556</td>
<td>$4,908</td>
<td>$5,450</td>
</tr>
<tr class="current-row">
<th>Roosevelt Island</th>
<td>$3,430</td>

### Create DataFrame

In [82]:
# get Neighborhood from table headers
table_headers = [th.getText() for th in mytable.findAll('th')]
table_headers_neigh = table_headers[7:]
df_col_neigh = pd.DataFrame({'Neighborhood':table_headers_neigh})

# get Studio Rent from table rows
table_data = []
table_rows = mytable.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    table_data.append(row)
# dataframe consists of five columns: All rentals, Studio Rent, 1 Bed , 2 Beds, and 3 Beds
df_data = pd.DataFrame(table_data,columns=['All rentals','Studio Rent','1 Bed','2 Beds','3 Beds'])
df_data.drop([0,1],inplace=True)
# only keep 'Studio Rent' column
df_data.drop(columns = ['All rentals','1 Bed','2 Beds','3 Beds'], inplace=True)
df_data.reset_index(drop=True, inplace=True)
df_data['Studio Rent'] = df_data['Studio Rent'].str.replace(',', '')
df_data['Studio Rent'] = df_data['Studio Rent'].str.replace('$', '')
df_data['Studio Rent'] = df_data['Studio Rent'].astype(int)

# Create DataFrame consisting Neighborhoods and their average Studio Rent price
df_rent = pd.concat([df_col_neigh, df_data], axis=1)
df_rent

,Neighborhood,Studio Rent
0,Washington Heights,1678
1,East Harlem,1895
2,Harlem,1975
3,Ellis Island,2929
4,Tudor City,2967
5,Roosevelt Island,3192
6,Stuyvesant Town,2632
7,Central Park South,2994
8,Theatre District - Times Square,2981
9,Central Midtown,2901


### (2) Geographic Data

We will analyze and compare Flushing and the neighborhoods in Manhattan. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains Flushing, Queens and the neighborhoods that exist in Manhattan as well as the latitude and longitude coordinates of each neighborhood. 

The files can be downloaded from IBM Cloud by run  a `wget` command. And then do some data wrangling and transform the data into dataframe with 4 columns: **['Borough', 'Neighborhood', 'Latitude', 'Longitude']**

In [83]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('NYC Data downloaded!')

NYC Data downloaded!


#### Load and explore the data

In [84]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [85]:
neighborhoods_data = newyork_data['features']

In [86]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

#### Quickly examine the resulting dataframe

In [87]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Let's examine how many neighborhoods in each borough are

In [88]:
neighborhoods['Borough'].value_counts()

Queens           81
Brooklyn         70
Staten Island    63
Bronx            52
Manhattan        40
Name: Borough, dtype: int64

#### Keep only the **'Flushing, Queens, NY'** and the **neighborhoods in Manhattan**. 
#### So let's slice the original dataframe and create a new dataframe of Flushing and the Manhattan data.

In [89]:
fl_data = neighborhoods[neighborhoods['Neighborhood'] == 'Flushing']
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
fl_manhattan_data = pd.concat([fl_data,manhattan_data]).reset_index(drop=True)
fl_manhattan_data

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Flushing,40.764454,-73.831773
1,Manhattan,Marble Hill,40.876551,-73.910660
2,Manhattan,Chinatown,40.715618,-73.994279
3,Manhattan,Washington Heights,40.851903,-73.936900
4,Manhattan,Inwood,40.867684,-73.921210
5,Manhattan,Hamilton Heights,40.823604,-73.949688
6,Manhattan,Manhattanville,40.816934,-73.957385
7,Manhattan,Central Harlem,40.815976,-73.943211
8,Manhattan,East Harlem,40.792249,-73.944182
9,Manhattan,Upper East Side,40.775639,-73.960508


#### Let's get the geographical coordinates of Manhattan.

In [90]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


#### Let's visualize Flushing and the neighborhoods in Manhattan.

In [91]:
# create map of Manhattan using latitude and longitude values
map_fl_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(fl_manhattan_data['Latitude'], fl_manhattan_data['Longitude'], fl_manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_fl_manhattan)  
    
map_fl_manhattan

### (3) Foursquare Location Data
Utilizing **Foursquare API** to explore neighborhoods and segment them.
#### Define Foursquare Credentials and Version

In [92]:
CLIENT_ID = 'TCYPPRL3BT41PAGLPZBCKOQTRUTJNVISGZMJSYRCVR1NSR2I'
CLIENT_SECRET = 'AWH4400LSJVSZJKSTVQW0RSQTELTR3KCHKTJVMQ5UESYZJCO'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TCYPPRL3BT41PAGLPZBCKOQTRUTJNVISGZMJSYRCVR1NSR2I
CLIENT_SECRET:AWH4400LSJVSZJKSTVQW0RSQTELTR3KCHKTJVMQ5UESYZJCO


In [93]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## 2. Explore Neighborhoods in Manhattan

Let's create a function by using Foursquare API to get top 100 venues that are in Flushing and the neighborhoods in Manhattan within a radius of 500 meters, and transform the data into dataframe with 7 columns: **[‘Neighborhood', ‘Neighborhood Latitude', ‘Neighborhood Longitude', ‘Venue’, ‘Venue Latitude', ‘Venue Longitude', ‘Venue Category’]**

In [94]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *fl_manhattan_venues*.

In [96]:
fl_manhattan_venues = getNearbyVenues(names=fl_manhattan_data['Neighborhood'],
                                   latitudes=fl_manhattan_data['Latitude'],
                                   longitudes=fl_manhattan_data['Longitude']
                                  )

Flushing
Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


#### Let's check the size of the resulting dataframe.

In [97]:
print(fl_manhattan_venues.shape)
fl_manhattan_venues.head()

(3360, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Flushing,40.764454,-73.831773,Debasaki,40.764852,-73.831767,Korean Restaurant
1,Flushing,40.764454,-73.831773,Hunan House Chinese Restaurant,40.763450,-73.829818,Chinese Restaurant
2,Flushing,40.764454,-73.831773,Flushing Town Hall,40.763623,-73.830139,Performing Arts Venue
3,Flushing,40.764454,-73.831773,Little Sheep Mongolian Hot Pot 小肥羊,40.762083,-73.828963,Hotpot Restaurant
4,Flushing,40.764454,-73.831773,Joe's Shanghai 鹿鸣春,40.761794,-73.830341,Dumpling Restaurant


Let's check how many venues were returned for each neighborhood

In [98]:
fl_manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,94,94,94,94,94,94
Carnegie Hill,100,100,100,100,100,100
Central Harlem,45,45,45,45,45,45
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,42,42,42,42,42,42
East Village,100,100,100,100,100,100


Let's find out how many unique categories can be curated from all the returned venues.

In [99]:
print('There are {} uniques categories.'.format(len(fl_manhattan_venues['Venue Category'].unique())))

There are 330 uniques categories.


## 3. Analyze Each Neighborhood

In [100]:
# one hot encoding
fl_manhattan_onehot = pd.get_dummies(fl_manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
fl_manhattan_onehot['Neighborhood'] = fl_manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [fl_manhattan_onehot.columns[-1]] + list(fl_manhattan_onehot.columns[:-1])
fl_manhattan_onehot = fl_manhattan_onehot[fixed_columns]

fl_manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul 

Let's examine the new dataframe size.

In [101]:
fl_manhattan_onehot.shape

(3360, 331)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [102]:
fl_manhattan_grouped = fl_manhattan_onehot.groupby('Neighborhood').mean().reset_index()
fl_manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul 

#### Let's print each neighborhood along with the top 5 most common venues

In [103]:
num_top_venues = 5

for hood in fl_manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = fl_manhattan_grouped[fl_manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
         venue  freq
0  Coffee Shop  0.09
1         Park  0.07
2        Hotel  0.05
3          Gym  0.03
4    Wine Shop  0.03


----Carnegie Hill----
            venue  freq
0     Pizza Place  0.06
1  Cosmetics Shop  0.05
2     Coffee Shop  0.05
3            Café  0.04
4             Bar  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1   Chinese Restaurant  0.07
2           Public Art  0.04
3   Seafood Restaurant  0.04
4  American Restaurant  0.04


----Chelsea----
                venue  freq
0  Italian Restaurant  0.06
1         Coffee Shop  0.06
2           Nightclub  0.04
3              Bakery  0.04
4      Ice Cream Shop  0.04


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.09
1    American Restaurant  0.04
2     Dim Sum Restaurant  0.04
3  Vietnamese Restaurant  0.04
4        Bubble Tea Shop  0.04


----Civic Center----
                  venue  freq
0  Gym / Fitness Center  0.06
1     

#### Let's put that into a *pandas dataframe*

First, let's write a function to sort the venues in descending order.

In [104]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [105]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = fl_manhattan_grouped['Neighborhood']

for ind in np.arange(fl_manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(fl_manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Wine Shop,Italian Restaurant,Gym,Ice Cream Shop,Market,Fountain,Burger Joint
1,Carnegie Hill,Pizza Place,Coffee Shop,Cosmetics Shop,Café,Yoga Studio,Italian Restaurant,Bookstore,Bar,French Restaurant,Japanese Restaurant
2,Central Harlem,African Restaurant,Chinese Restaurant,Seafood Restaurant,American Restaurant,Cosmetics Shop,French Restaurant,Public Art,Clothing Store,Gym,Library
3,Chelsea,Italian Restaurant,Coffee Shop,Nightclub,Ice Cream Shop,Bakery,Art Gallery,Seafood Restaurant,Theater,American Restaurant,Hotel
4,Chinatown,Chinese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,American Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Salon / Barbershop,Bakery,Bar,Ice Cream Shop
5,Civic Center,Gym / Fitness Center,Cocktail Bar,Bakery,Italian Restaurant,Park,Spa,Sporting Goods Shop,Gym,French Restaurant,Sandwich Place
6,Clinton,Theater,Italian Restaurant,Hotel,American Restaurant,Wine Shop,Coffee Shop,Indie Theater,Gym / Fitness Center,Gym,Spa
7,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Thai Restaurant,Deli / Bodega,Spa,Beer Bar,Taco Place,French Restaurant,Street Art
8,East Village,Ice Cream Shop,Bar,Wine Bar,Mexican Restaurant,Chinese Restaurant,Cocktail Bar,Pizza Place,Coffee Shop,Vegetarian / Vegan Restaurant,Italian Restaurant
9,Financial District,Coffee Shop,Hotel,Steakhouse,Wine Shop,Food Truck,Italian Restaurant,Bar,Pizza Place,Juice Bar,Park


## 4. Cluster Neighborhoods --- **using k-means clustering**

In [106]:
# set number of clusters
kclusters = 4

fl_manhattan_grouped_clustering = fl_manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(fl_manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 1, 3, 1, 1, 3, 0, 1, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [107]:
fl_manhattan_merged = fl_manhattan_data

# add clustering labels
fl_manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
fl_manhattan_merged = fl_manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

fl_manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Flushing,40.764454,-73.831773,3,Karaoke Bar,Bubble Tea Shop,Chinese Restaurant,Korean Restaurant,Bakery,Hotpot Restaurant,Szechuan Restaurant,Seafood Restaurant,Asian Restaurant,Construction & Landscaping
1,Manhattan,Marble Hill,40.876551,-73.910660,1,Discount Store,Sandwich Place,Yoga Studio,Comfort Food Restaurant,Supplement Shop,Steakhouse,Shoe Store,Seafood Restaurant,Coffee Shop,Pizza Place
2,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,American Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Salon / Barbershop,Bakery,Bar,Ice Cream Shop
3,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Sandwich Place,Mobile Phone Shop,Bakery,Mexican Restaurant,Chinese Restaurant,Caribbean Restaurant,Park,Clothing Store,Supermarket
4,Manhattan,Inwood,40.867684,-73.921210,1,Mexican Restaurant,Café,Lounge,Pizza Place,Wine Bar,Bakery,Frozen Yogurt Shop,American Restaurant,Park,Restaurant


Finally, let's visualize the resulting clusters

In [108]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(fl_manhattan_merged['Latitude'], fl_manhattan_merged['Longitude'], fl_manhattan_merged['Neighborhood'], fl_manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters and Find Similar Neighborhoods in Manhattan

We can examine each cluster and determine the discriminating venue categories that distinguish each cluster.
#### Cluster 1

In [109]:
fl_manhattan_merged.loc[fl_manhattan_merged['Cluster Labels'] == 0, fl_manhattan_merged.columns[[1] + list(range(5, fl_manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Harlem,African Restaurant,Chinese Restaurant,Seafood Restaurant,American Restaurant,Cosmetics Shop,French Restaurant,Public Art,Clothing Store,Gym,Library
14,Lincoln Square,Theater,Gym / Fitness Center,Plaza,Concert Hall,Italian Restaurant,French Restaurant,Performing Arts Venue,Indie Movie Theater,Bakery,Café
16,Midtown,Hotel,Coffee Shop,Food Truck,Steakhouse,Theater,Park,Bookstore,Spa,Sandwich Place,Clothing Store
23,Little Italy,Bakery,Café,Yoga Studio,Cocktail Bar,Bubble Tea Shop,Italian Restaurant,Seafood Restaurant,Salon / Barbershop,Ice Cream Shop,Chinese Restaurant
29,Battery Park City,Coffee Shop,Park,Hotel,Wine Shop,Italian Restaurant,Gym,Ice Cream Shop,Market,Fountain,Burger Joint
34,Midtown South,Korean Restaurant,Coffee Shop,Hotel Bar,Cocktail Bar,Gym / Fitness Center,Boutique,Bakery,Hotel,Cosmetics Shop,Japanese Restaurant
38,Stuyvesant Town,Bar,Park,Boat or Ferry,Heliport,Farmers Market,Basketball Court,Baseball Field,Beer Garden,Cocktail Bar,Coffee Shop


#### Cluster 2

In [110]:
fl_manhattan_merged.loc[fl_manhattan_merged['Cluster Labels'] == 1, fl_manhattan_merged.columns[[1] + list(range(5, fl_manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Marble Hill,Discount Store,Sandwich Place,Yoga Studio,Comfort Food Restaurant,Supplement Shop,Steakhouse,Shoe Store,Seafood Restaurant,Coffee Shop,Pizza Place
2,Chinatown,Chinese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,American Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Salon / Barbershop,Bakery,Bar,Ice Cream Shop
4,Inwood,Mexican Restaurant,Café,Lounge,Pizza Place,Wine Bar,Bakery,Frozen Yogurt Shop,American Restaurant,Park,Restaurant
5,Hamilton Heights,Mexican Restaurant,Coffee Shop,Café,Pizza Place,Bakery,Deli / Bodega,Cocktail Bar,Sandwich Place,School,Chinese Restaurant
8,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Thai Restaurant,Deli / Bodega,Spa,Beer Bar,Taco Place,French Restaurant,Street Art
10,Yorkville,Italian Restaurant,Bar,Coffee Shop,Gym,Sushi Restaurant,Pizza Place,Japanese Restaurant,Deli / Bodega,Ice Cream Shop,Gym / Fitness Center
11,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Gym / Fitness Center,Gym,Burger Joint,Deli / Bodega,Turkish Restaurant,Bakery
13,Upper West Side,Italian Restaurant,Bar,Coffee Shop,Wine Bar,Indian Restaurant,Bakery,Burger Joint,Vegetarian / Vegan Restaurant,French Restaurant,Pub
17,Murray Hill,Japanese Restaurant,Hotel,Italian Restaurant,Gym,Bar,Sandwich Place,Salon / Barbershop,Coffee Shop,French Restaurant,Gym / Fitness Center
19,Greenwich Village,Italian Restaurant,Sushi Restaurant,Clothing Store,French Restaurant,Café,Bakery,American Restaurant,Chinese Restaurant,Indian Restaurant,Seafood Restaurant


#### Cluster 3

In [111]:
fl_manhattan_merged.loc[fl_manhattan_merged['Cluster Labels'] == 2, fl_manhattan_merged.columns[[1] + list(range(5, fl_manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Carnegie Hill,Pizza Place,Coffee Shop,Cosmetics Shop,Café,Yoga Studio,Italian Restaurant,Bookstore,Bar,French Restaurant,Japanese Restaurant


#### Cluster 4

In [112]:
fl_manhattan_merged.loc[fl_manhattan_merged['Cluster Labels'] == 3, fl_manhattan_merged.columns[[1] + list(range(5, fl_manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Flushing,Karaoke Bar,Bubble Tea Shop,Chinese Restaurant,Korean Restaurant,Bakery,Hotpot Restaurant,Szechuan Restaurant,Seafood Restaurant,Asian Restaurant,Construction & Landscaping
3,Washington Heights,Café,Sandwich Place,Mobile Phone Shop,Bakery,Mexican Restaurant,Chinese Restaurant,Caribbean Restaurant,Park,Clothing Store,Supermarket
6,Manhattanville,Deli / Bodega,Mexican Restaurant,Coffee Shop,Italian Restaurant,Seafood Restaurant,Music School,Bike Trail,Sushi Restaurant,Supermarket,Burger Joint
9,Upper East Side,Italian Restaurant,Exhibit,Coffee Shop,Juice Bar,Bakery,Art Gallery,Spa,Gym / Fitness Center,French Restaurant,Boutique
12,Roosevelt Island,Park,Sandwich Place,Liquor Store,Café,Market,Greek Restaurant,School,Gym,Coffee Shop,Metro Station
15,Clinton,Theater,Italian Restaurant,Hotel,American Restaurant,Wine Shop,Coffee Shop,Indie Theater,Gym / Fitness Center,Gym,Spa
18,Chelsea,Italian Restaurant,Coffee Shop,Nightclub,Ice Cream Shop,Bakery,Art Gallery,Seafood Restaurant,Theater,American Restaurant,Hotel
20,East Village,Ice Cream Shop,Bar,Wine Bar,Mexican Restaurant,Chinese Restaurant,Cocktail Bar,Pizza Place,Coffee Shop,Vegetarian / Vegan Restaurant,Italian Restaurant
24,Soho,Clothing Store,Women's Store,Shoe Store,Boutique,Men's Store,Italian Restaurant,Coffee Shop,Mediterranean Restaurant,Spa,Art Gallery
26,Manhattan Valley,Pizza Place,Coffee Shop,Bar,Yoga Studio,Mexican Restaurant,Indian Restaurant,Italian Restaurant,Playground,American Restaurant,Café


### As we can see that Flushing is in Cluster 4.
#### Let's create a dataframe that includes all the neighborhoods in Manhattan in Cluster 4.

In [113]:
cluster4 = fl_manhattan_merged.loc[fl_manhattan_merged['Cluster Labels'] == 3][['Neighborhood']]
manhattan_cluster4 = cluster4.loc[cluster4['Neighborhood'] != 'Flushing'].reset_index(drop=True)
print("The similar neighborhoods in Manhattan are:")
manhattan_cluster4

The similar neighborhoods in Manhattan are:


,Neighborhood
0,Washington Heights
1,Manhattanville
2,Upper East Side
3,Roosevelt Island
4,Clinton
5,Chelsea
6,East Village
7,Soho
8,Manhattan Valley


## 6. Rank the Similar Neighborhoods by Studio Rent Price

#### Studio Rent by Neighborhood in Manhattan

In [114]:
df_rent

,Neighborhood,Studio Rent
0,Washington Heights,1678
1,East Harlem,1895
2,Harlem,1975
3,Ellis Island,2929
4,Tudor City,2967
5,Roosevelt Island,3192
6,Stuyvesant Town,2632
7,Central Park South,2994
8,Theatre District - Times Square,2981
9,Central Midtown,2901


#### Studio Rent of Similar Neighborhoods ranking in ascending order

In [115]:
pd.concat([manhattan_cluster4, df_rent], axis=1, join='inner')
cluster4_rent = pd.merge(manhattan_cluster4, df_rent,  how='left', left_on=['Neighborhood'], right_on = ['Neighborhood']).dropna()
cluster4_rent_asc = cluster4_rent.sort_values('Studio Rent', axis=0, ascending = 'True').reset_index(drop=True)
cluster4_rent_asc

,Neighborhood,Studio Rent
0,Washington Heights,1678.0
1,Manhattan Valley,2506.0
2,East Village,2933.0
3,Chelsea,3171.0
4,Roosevelt Island,3192.0


### Find the Similar neighborhood in Manhattan as Flushing with Lowest Studio Rental price

In [116]:
print('The similar neighborhood in Manhattan as Flushing with Lowest Studio Rental Price: ', cluster4_rent_asc.iloc[0]['Neighborhood'])
print('The Average Rental Price in ', cluster4_rent_asc.iloc[0]['Neighborhood'], ': $', cluster4_rent_asc.iloc[0]['Studio Rent'])

The similar neighborhood in Manhattan as Flushing with Lowest Studio Rental Price:  Washington Heights
The Average Rental Price in  Washington Heights : $ 1678.0
